In [ ]:
import sys
import sklearn
import numpy as np
import os
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
import random

In [ ]:
colnames = ["duration","protocol_type","service","flag","src_bytes","dst_bytes","land","wrong_fragment","urgent","hot","num_failed_logins","logged_in","num_compromised","root_shell","su_attempted","num_root","num_file_creations","num_shells","num_access_files","num_outbound_cmds","is_host_login","is_guest_login","count","srv_count","serror_rate","srv_serror_rate","same_srv_rate","diff_srv_rate","srv_diff_host_rate","una1","una2","dst_host_count","dst_host_srv_count","dst_host_same_srv_rate","dst_host_diff_srv_rate","dst_host_same_src_port_rate","dst_host_srv_diff_host_rate","dst_host_serror_rate","dst_host_srv_serror_rate","dst_host_rerror_rate","dst_host_srv_rerror_rate","result"]
# len(colnames)

In [ ]:
import pandas as pd
from zipfile import ZipFile
from urllib.request import urlopen
import io

f = urlopen('https://raw.githubusercontent.com/AkshatGarg7/DDOS-Detection-using-Machine-Learning/master/Dataset/dataset.zip').read()
zip_file = ZipFile(io.BytesIO(f))

train_df = pd.read_csv(zip_file.open('kddcup.csv'),header = None,names = colnames)
print("train_df shape -> ",train_df.shape)
test_df = pd.read_csv(zip_file.open('corrected.csv'),header = None,names = colnames)
print("test_df shape -> ",test_df.shape)

train_df shape ->  (494021, 42)
test_df shape ->  (311029, 42)


In [ ]:
train_df.head()

duration protocol_type  ... dst_host_srv_rerror_rate   result
0         0           tcp  ...                      0.0  normal.
1         0           tcp  ...                      0.0  normal.
2         0           tcp  ...                      0.0  normal.
3         0           tcp  ...                      0.0  normal.
4         0           tcp  ...                      0.0  normal.

[5 rows x 42 columns]

In [ ]:
test_df.head()

duration protocol_type  ... dst_host_srv_rerror_rate          result
0         0           udp  ...                      0.0         normal.
1         0           udp  ...                      0.0         normal.
2         0           udp  ...                      0.0         normal.
3         0           udp  ...                      0.0  snmpgetattack.
4         0           udp  ...                      0.0  snmpgetattack.

[5 rows x 42 columns]

In [ ]:
train_df.isnull().sum() 
#so no data is null in train_df

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
una1                           0
una2      

In [ ]:
test_df.isnull().sum()
#so no data is null in test_df

duration                       0
protocol_type                  0
service                        0
flag                           0
src_bytes                      0
dst_bytes                      0
land                           0
wrong_fragment                 0
urgent                         0
hot                            0
num_failed_logins              0
logged_in                      0
num_compromised                0
root_shell                     0
su_attempted                   0
num_root                       0
num_file_creations             0
num_shells                     0
num_access_files               0
num_outbound_cmds              0
is_host_login                  0
is_guest_login                 0
count                          0
srv_count                      0
serror_rate                    0
srv_serror_rate                0
same_srv_rate                  0
diff_srv_rate                  0
srv_diff_host_rate             0
una1                           0
una2      

In [ ]:
pd.Series(train_df.protocol_type.value_counts(ascending=True))

udp      20354
tcp     190065
icmp    283602
Name: protocol_type, dtype: int64

In [ ]:
pd.Series(test_df.protocol_type.value_counts(ascending=True))

udp      26703
tcp     119357
icmp    164969
Name: protocol_type, dtype: int64

In [ ]:
pd.Series(train_df.result.value_counts(ascending=True))

spy.                     2
perl.                    3
phf.                     4
multihop.                7
ftp_write.               8
loadmodule.              9
rootkit.                10
imap.                   12
warezmaster.            20
land.                   21
buffer_overflow.        30
guess_passwd.           53
nmap.                  231
pod.                   264
teardrop.              979
warezclient.          1020
portsweep.            1040
ipsweep.              1247
satan.                1589
back.                 2203
normal.              97278
neptune.            107201
smurf.              280790
Name: result, dtype: int64

In [ ]:
pd.Series(test_df.result.value_counts(ascending=True))

imap.                    1
worm.                    2
sqlattack.               2
loadmodule.              2
perl.                    2
udpstorm.                2
phf.                     2
ftp_write.               3
xsnoop.                  4
xlock.                   9
land.                    9
teardrop.               12
rootkit.                13
xterm.                  13
ps.                     16
named.                  17
sendmail.               17
multihop.               18
buffer_overflow.        22
nmap.                   84
pod.                    87
httptunnel.            158
ipsweep.               306
portsweep.             354
saint.                 736
processtable.          759
apache2.               794
mscan.                1053
back.                 1098
warezmaster.          1602
satan.                1633
snmpguess.            2406
guess_passwd.         4367
mailbomb.             5000
snmpgetattack.        7741
neptune.             58001
normal.              60593
s

In [ ]:
features = ["duration","protocol_type","src_bytes","wrong_fragment","count","urgent","num_compromised","srv_count"]
target = "result"

In [ ]:
# X = train_df.loc[:,features]
# y = test_df.loc[:,target]

In [ ]:
# classes = np.unique(y)
# print(classes)

In [ ]:
train_df["result"] = train_df["result"].replace(["normal."],1)
train_df["result"] = train_df["result"].replace(['back.','buffer_overflow.','ftp_write.','guess_passwd.','imap.','ipsweep.','land.','loadmodule.','multihop.','neptune.','nmap.','perl.','phf.','pod.','portsweep.','rootkit.','satan.','smurf.','spy.','teardrop.','warezclient.','warezmaster.'],0)

In [ ]:
test_df["result"] = test_df["result"].replace(["normal."],1)
test_df["result"] = test_df["result"].replace(['apache2.','back.','buffer_overflow.','ftp_write.','guess_passwd.','httptunnel.','imap.','ipsweep.','land.','loadmodule.','mailbomb.','mscan.','multihop.','named.','neptune.','nmap.','perl.','phf.','pod.','portsweep.','processtable.','ps.','rootkit.','saint.','satan.','sendmail.','smurf.','snmpgetattack.','snmpguess.','sqlattack.','teardrop.','udpstorm.','warezmaster.','worm.','xlock.','xsnoop.','xterm.'],0)

In [ ]:
pd.Series(train_df.result.value_counts(ascending=True))

1     97278
0    396743
Name: result, dtype: int64

In [ ]:
pd.Series(test_df.result.value_counts(ascending=True))

1     60593
0    250436
Name: result, dtype: int64